In [1]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
from src.data import *
from src.space import *
from src.mcmc import *

# https://dvr.capitol.texas.gov/
level    = 'cntyvtd'
proposal = 'plans2100'  # for 2010 enacted, use 'planc2100' for US congress, 'plans2100' for TX Senate, or 'planh2100' for TX House
contract = 0
# contract = 'proposal'

# D = Data()
S = Space(level=level, proposal=proposal, contract=contract)
M = MCMC(gpickle=S.gpickle, nodes=S.tbls['nodes'],
         max_steps   = 5,
         random_seed = 0,
         defect_cap  = 1000000,
         pop_deviation_target=200)
M.run_chain()
# M.post_process()

/home/jupyter/MathVGerrmandering_CMAT_2021


getting states


Get proposal    ... using existing table ... success!
Get nodes       ... processing ... using existing data table ... creating nodes table ... success!
Get graph       ... processing ... creating graph ... success!
random_seed 0: step 1 0hrs 0min 5.38sec, pop_deviation=32.7, intersect_defect=25, whole_defect=6, defect=31
random_seed 0: step 2 0hrs 0min 9.65sec, pop_deviation=30.9, intersect_defect=26, whole_defect=6, defect=32
random_seed 0: step 3 0hrs 0min 14.07sec, pop_deviation=64.7, intersect_defect=29, whole_defect=6, defect=35
random_seed 0: step 4 0hrs 0min 18.12sec, pop_deviation=64.7, intersect_defect=30, whole_defect=6, defect=36
random_seed 0: step 5 0hrs 0min 21.74sec, pop_deviation=64.7, intersect_defect=30, whole_defect=6, defect=36
random_seed 0: step 5 0hrs 0min 21.74sec, pop_deviation=64.7, intersect_defect=30, whole_defect=6, defect=36
saving ... plan_rec ... county_rec ... district_rec ... summary_rec ... done
post-processing
random_seed 0 done


In [2]:
try:
    import pandas_bokeh
except:
    os.system('pip install --upgrade pandas-bokeh')
    import pandas_bokeh
pandas_bokeh.output_notebook()

plan_cols = ['geoid', 'plan', 'district']
query = f"select {', '.join(plan_cols)} from {M.output}"
plans = run_query(query).pivot(index='geoid', columns='plan', values='district')

node_cols = ['geoid', 'polygon', 'county', 'total_pop', 'aland', 'density', 'polsby_popper']
query = f"select {', '.join(node_cols)} from {M.nodes}"
nodes = run_query(query).set_index('geoid')

Loading BokehJS ...

In [3]:
repl = {c:str(c) for c in plans.columns}
plans.rename(columns=repl, inplace=True)

geo = gpd.GeoSeries.from_wkt(nodes['polygon'], crs=crs_census).simplify(0.003).buffer(0) #<-- little white space @ .001 ~5.7 mb, minimal at .0001 ~10mb, with no white space ~37mb
gdf = nodes.drop(columns='polygon').join(plans, how='inner')
gdf = gpd.GeoDataFrame(gdf, geometry=geo).reset_index()
gdf.head(3)

,geoid,county,total_pop,aland,density,polsby_popper,0,1,2,3,4,5,geometry
0,025000011,Bee,366,86.810110,4.216099,48.26,21,21,21,21,21,21,"POLYGON ((-98.06778 28.67015, -97.91510 28.719..."
1,025000002,Bee,253,58.531673,4.322446,44.85,21,21,21,21,21,21,"POLYGON ((-97.94528 28.59626, -97.87002 28.621..."
2,025000012,Bee,324,19.075640,16.985013,38.15,21,21,21,21,21,21,"POLYGON ((-97.79267 28.60953, -97.76589 28.601..."


In [38]:
width = 600
xlim = [-106.2, -94.0]
ylim = [ 25.4 ,  36.6]
aspect = (ylim[1] - ylim[0]) / (xlim[1] - xlim[0])
height = round(aspect * width)
fn = str(S.gpickle)
fn = fn[:fn.rfind('_')] + '_animation.html'
print(f'saving animation to {fn}')

fig = gdf.plot_bokeh(
    colormap = "Category20",
    hovertool_string = '@geoid, @county<br>pop=@total_pop<br>density=@density{0.0}<br>land=@aland{0.0}<br>pp=@polsby_popper{0.0}',
    fill_alpha = 0.5,
    line_alpha = 0.05,
    return_html = True,
    show_figure = True,
    slider = list(repl.values()),
    slider_name = "PLAN",
    show_colorbar = False,
    xlim = xlim,
    ylim = ylim,
    figsize = (width, height),
)

with open(fn, 'w') as file:
    file.write(fig)

saving animation to /home/jupyter/redistricting_data/proposals/TX/sldl/planh2100/TX_sldl_planh2100_cntyvtd_0_animation.html
